In [ ]:
import numpy as np
from  numpy import deg2rad as d2r
from  numpy import array as arr

from scipy.integrate import solve_ivp, OdeSolution
from scipy.optimize import minimize, approx_fprime
from functools import lru_cache
from models import dynamics, CtrlMode, STANDARD_GRAV
from transcription import MultiShootingTranscription
from utils import unpack_sol_list
from plotting import plot, visualize_jac2

## Electron Rocket Parameters

In [ ]:
n_engines_s1 = 9
n_engines_s2 = 1
isp_s1 = 311.0
engine_thrust_s1 = n_engines_s1*24910.04  # N Average between sl and vac
isp_s2 = 343.0
engine_thrust_s2 = n_engines_s2* 25_000.0  # N
s1_vch_params = (engine_thrust_s1, isp_s1)
s2_vch_params = (engine_thrust_s2, isp_s2)

fairing_mass = 50.0
farinig_timing = 184.0 - 162.0 # sec
payload = 250.0
s1_dry_mass = 1076.47308279  
s2_dry_mass = 257.90093739  

s1_wet_mass = 10047.082106064723
s2_wet_mass = 2602.454913676189
total_mass = 12949.537019740912

mdot_s1 = engine_thrust_s1 / STANDARD_GRAV / isp_s1
mdot_s2 = engine_thrust_s2 / STANDARD_GRAV / isp_s2

In [ ]:
NUM_X= 5
NUM_U = 5
NUM_PHASE = 4
# %load_ext snakeviz

## Initial Guesses

In [ ]:
mu_earth = 3.986004418e14
earth_r = 6_378_000.0 # m
circ_orbit_alt = 200_000.0 
v_circ = np.sqrt(mu_earth / (earth_r + circ_orbit_alt))
# Guesses 
x0 = arr([
    [0,0,0,0,total_mass], 
    [7.5,390,1.5,80,12200],
    [30000,60000,2000,500,s2_wet_mass + payload + fairing_mass],
    [45000,80000,2800,1000,s2_wet_mass + payload - farinig_timing * mdot_s2]
])

x_f = arr([200_000, circ_orbit_alt, v_circ, 0.0, s2_dry_mass + payload])

## normalization vector 
u0_n_vec = arr([np.pi,0.1,np.pi/2,0.1,np.pi/2 ])
x0_n_vec = arr([circ_orbit_alt, circ_orbit_alt, 5000, 1000, 13000]*NUM_X)

t0_n_vec = arr([10,120,20,300])

normalization_vec = np.concatenate([u0_n_vec, x0_n_vec, t0_n_vec])

## Define a multiphase trajectory problem

In [ ]:
problem = MultiShootingTranscription(["phase0", "phase1", "phase2", "phase3"], NUM_X)

problem.set_dynamics_params("phase0", s1_vch_params)
problem.set_dynamics_params("phase1", s1_vch_params)
problem.set_dynamics_params("phase2", s2_vch_params)
problem.set_dynamics_params("phase3", s2_vch_params)

## Define state, control and time guesses for each phase 

In [ ]:
problem.set_phase_init_x("phase0", x0 = x0[0], bounds= x0[0])
problem.set_phase_control("phase0", CtrlMode.ANGLE_STEER , u0 = d2r(89.5), bounds = [(d2r(85), d2r(89.8))])
problem.set_phase_time("phase0", t0 = 10, bounds = 10)

problem.set_phase_init_x("phase1", x0 = x0[1])
problem.set_phase_control("phase1", CtrlMode.ZERO_ALPHA , u0 = [])
problem.set_phase_time("phase1", t0 = 100, bounds=(60, 180))

problem.set_phase_init_x("phase2", x0 = x0[2])
problem.set_phase_control("phase2", CtrlMode.LTS , u0 = arr([-0.001,1]), bounds = [(-0.1,0.1), (-3,3)])
problem.set_phase_time("phase2", t0 = farinig_timing, bounds = farinig_timing)

problem.set_phase_init_x("phase3", x0 = x0[3])
problem.set_phase_control("phase3", CtrlMode.LTS , u0 = arr([-0.001,1]), bounds = [(-0.1,0.1), (-3,3)])
problem.set_phase_time("phase3", t0 = 320)

problem.set_non_zero_defect(("phase1", "phase2"), arr([0,0,0,0,s1_dry_mass]))
problem.set_non_zero_defect(("phase2", "phase3"), arr([0,0,0,0,fairing_mass]))

problem.set_terminal_state(x_final = x_f, bounds = arr([None, circ_orbit_alt, v_circ, 0, None ]))

## Build The problem
Builds the decision vector and bounds 

In [ ]:
d0, d_bounds, full_params = problem.build()
d0_norm, d_bounds_norm = problem.normalize_decision_vec(d0, d_bounds,normalization_vec)

In [ ]:
# denormalize_decision_vec(d0_norm, normalization_vec)

## Defining dynamic constraint function

In [ ]:
def unpack_decision_var(decision_var, config, phase_id):
    """Converts the decision 

    Args:
        decision_var : Optimzation decission vector
        config : Config for this phase
        phase_id : id for the phase

    Returns:
        tuple: control, state, terminal time, control_law

    """
    control_law = config[0]
    u = decision_var[:NUM_U][range(*config[1])]
    x = decision_var[NUM_U:(NUM_PHASE+1)*NUM_X + NUM_U].reshape(NUM_PHASE+1, NUM_X)[phase_id]
    t_terminal = decision_var[-NUM_PHASE:][phase_id] # Time is the last decision var here
    return (u, x, t_terminal, control_law)


@lru_cache(maxsize=128, typed=True) 
def traj_rollout(t_terminal:float, x0: np.array, params: tuple) -> OdeSolution:
    """Integrates a phase of the trajectory.
    The trajectory is evaluated at a set time points using t_eval, this greatly improves convergance and stability of the gradients 
    lru_cache decerases the time requre to calculate the jac, since scipy uses forward diff the cached f(x) used instead of a re-compute
    Args:
        t_terminal : Terminal time of the phase
        x0 : Initial state of the phase
        params : Phase Parameter

    Returns:
        OdeSolution: The solution of the phase
    """
    sol = solve_ivp(
        dynamics, 
        t_span=[0.0, t_terminal], 
        t_eval= np.linspace(0.0, t_terminal), # This greatly improves convergance and stability of the jac
        y0=x0,    
        args=(params,)
    )
    return sol  

def full_traj_rollout(d0, config_list):
    sol_list = []
    for  phase_id, config in enumerate(config_list):
        u, x, t_terminal, control_law = unpack_decision_var(d0,config, phase_id=phase_id)
        # make inputs hashable, neede for lru cache, the copy is cheaper than a second f(x) eval
        u_ = tuple(u.tolist())
        x_ =tuple(x.tolist())
        t_ = float(t_terminal)
        
        vch_params = (config[3],(control_law, u_))
        #if sol is not None:
        #    x = sol.y[:,-1]
        sol = traj_rollout(t_, x_, vch_params)
        sol_list.append(sol)
    return sol_list

def dynamics_knot_constrant(d0_in, config_list): 
    d0 = problem.denormalize_decision_vec(d0_in, normalization_vec)
    defect_vector_list = []
    sol_list=  full_traj_rollout(d0, config_list)
    for idx in range(1,NUM_PHASE):
        _,_, knot_defect,_ = config_list[idx]
        defect_sub_vector = sol_list[idx].y[:,0] - sol_list[idx-1].y[:,-1] + knot_defect
        defect_sub_vector /= arr([100000, 100000, 8000, 1000, 1000])
        defect_vector_list.append(defect_sub_vector)
    # Terminal Defect 
    terminal_state = d0[NUM_U:(NUM_PHASE+1)*NUM_X + NUM_U].reshape(NUM_PHASE+1, NUM_X)[-1]
    terminal_defect = terminal_state - sol_list[-1].y[:,-1]
    terminal_defect /= arr([10000, 10000, 1000, 1000, 1000])
    defect_vector_list.append(terminal_defect)
    defect_vec = arr(defect_vector_list).flatten()
    return defect_vec

In [ ]:
sol_list = full_traj_rollout(d0, full_params)

In [ ]:
constraints = [{'type': 'eq', 'fun': dynamics_knot_constrant, 'args':(full_params,) },]

## Objective function
Maximize stage 2 mass

In [ ]:
def objective(decision_var: tuple, params: tuple) -> float:
        """Objective function for min prop

        Args:
            decision_var : Optimization problem decision vector
            params : 

        Returns:
            Cost to minimize
        """
        x = decision_var[NUM_U:(NUM_PHASE+1)*NUM_X + NUM_U].reshape(NUM_PHASE+1, NUM_X)
        terminal_state = x[-1]
        terminal_mass= terminal_state[-1]
        return -terminal_mass*terminal_mass*1000


def jac_objective(decision_var: tuple, params: tuple):
        jac = np.zeros_like(decision_var)
        val = -decision_var[29] - decision_var[29]
        jac[29]= val*1000
        return jac

## Scipy Minimize
SLSQP has to be use here because it can handle bounds and equality constraints.

In [ ]:
# %%snakeviz

result = minimize(
    objective, 
    d0_norm, 
    jac= jac_objective,
    method='SLSQP', 
    bounds=d_bounds_norm, 
    constraints=constraints,
    options = {"maxiter": 500, "disp": True},
    args=(full_params,)
)

In [ ]:
constraint_jac = approx_fprime(
    result.x, 
    dynamics_knot_constrant, 
    np.float64(1.4901161193847656e-08), full_params)

In [ ]:
visualize_jac2(result.x, constraint_jac)

In [ ]:
x_opt = problem.denormalize_decision_vec(result.x, normalization_vec)

sol_list = full_traj_rollout(x_opt, full_params)

In [ ]:
full_params

In [ ]:
plot(
    *unpack_sol_list(sol_list,0),
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Pos x",
    trace_names=("phase0", "phase1", "phase2", "phase3")
    )

In [ ]:
plot(
    *unpack_sol_list(sol_list,1),
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Pos y",
    trace_names=("phase0", "phase1", "phase2", "phase3")
    )


In [ ]:
plot(
    *unpack_sol_list(sol_list,2),
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Vel",
    trace_names=("phase0", "phase1", "phase2", "phase3")
    )

In [ ]:
plot(
    *unpack_sol_list(sol_list,3),
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Vel",
    trace_names=("phase0", "phase1", "phase2", "phase3")
    )

In [ ]:
plot(
    *unpack_sol_list(sol_list,4),
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Mass",
    trace_names=("phase0", "phase1", "phase2", "phase3")
    )

In [ ]:
times, state= unpack_sol_list(sol_list,4)

In [ ]:
unpack_decision_var(x_opt, full_params[3],3 )[0]

In [ ]:
from models import lts_control

In [ ]:
lts_2 =lts_control(times[2] - times[2][0], 0, unpack_decision_var(x_opt, full_params[2],2 )[0])
lts_3 =lts_control(times[3]- times[3][0], 0, unpack_decision_var(x_opt, full_params[3],3 )[0])


In [ ]:
plot(
    [times[2], times[3]],
    [lts_2, lts_3],
    title="Time vs States", 
    xlabel="Time", 
    ylabel="Pitch",
    trace_names=("phase2", "phase3")
    )